In [1]:
from dbfread import DBF
import pandas as pd
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

dbf = DBF('../data/Municipalities_with_topo.dbf', encoding='utf-8')
df = pd.DataFrame(iter(dbf))
df.to_csv("../data/Municipalities_with_topo.csv")


In [ ]:
df.columns

In [ ]:
vars = df.columns[5:]
vars

In [10]:
px.strip(df, x="tri1k_mean", hover_name="NOMGEO", color="CVE_ENT",height=500)

In [ ]:
# Matrix correlations
fig_1 = px.scatter_matrix(df,
                dimensions=['tri1k_mean', 'vrm1k_mean', 'rou1k_mean','slp1k_mean'],
                hover_name="NOMGEO",
                width=800,
                height=500,
                title="<b>Matrix correlations<b>",
                opacity=.6,
                )

fig_1.update_traces(diagonal_visible=False)
fig_1.show()
fig_1.write_image("D:/Documentos/Banco Mundial/do/DESARROLLO DIGITAL/LAC_MOBILE/images/mat_corr.png")

In [35]:
import pandas as pd
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import os

# Read data
df = pd.read_csv('../data/Municipalities_with_topo.csv')

app = Dash(__name__)
server = app.server

vars = df.columns[5:]

# Lista de variables con etiquetas
variable_options = [
    {'label': 'Tri Mean', 'value': 'tri1k_mean'},
    {'label': 'VRM Mean', 'value': 'vrm1k_mean'},
    {'label': 'ROU Mean', 'value': 'rou1k_mean'},
    {'label': 'SLP Mean', 'value': 'slp1k_mean'},
    {'label': 'Tri Sd', 'value': 'tri1k_sd'},
    {'label': 'VRM Sd', 'value': 'vrm1k_sd'},
    {'label': 'ROU Sd', 'value': 'rou1k_sd'},
    {'label': 'SLP Sd', 'value': 'slp1k_sd'},
    # Añadir más variables y etiquetas aquí según sea necesario
]

# Diccionario de etiquetas
label_map = {option['value']: option['label'] for option in variable_options}

app.layout = html.Div([
    html.H4('Matrix Correlations'),
    html.Div([
        html.Label('Select Variables:'),
        dcc.Dropdown(
            id="dropdown",
            options=variable_options,
            value=['tri1k_mean', 'vrm1k_mean', 'rou1k_mean', 'slp1k_mean'],
            multi=True,
            style={'width': '80%'}
        ),
    ], style={'width': '100%', 'display': 'flex', 'justify-content': 'center', 'margin-bottom': '20px'}),
    
    html.Div([
        html.H5('Scatter Matrix Plot'),
        dcc.Graph(id="scatter_matrix", style={'height': '70vh'})
    ], style={'width': '100%', 'display': 'flex', 'justify-content': 'center', 'flexDirection': 'column', 'align-items': 'center'}),

    html.Div([
        html.H5('Strip Plot'),
        dcc.Graph(id="strip_plot", style={'height': '70vh'})
    ], style={'width': '100%', 'display': 'flex', 'justify-content': 'center', 'flexDirection': 'column', 'align-items': 'center'})
])


@app.callback(
    [Output("scatter_matrix", "figure"), Output("strip_plot", "figure")], 
    [Input("dropdown", "value")]
)
def update_bar_chart(dims):
    fig = px.scatter_matrix(
        df, 
        dimensions=dims,
        labels={dim: label_map[dim] for dim in dims},
        height=500,
        width=800
    )
    fig.update_traces(diagonal_visible=False)
    
    fig2 = px.strip(df,
                    x=dims,
                    hover_name="NOMGEO",
                    color="CVE_ENT",
                    height=500,
                    width=800
    )
    return fig,fig2

app.run(debug=True,port=8051)

# PARA MAPAS

In [ ]:
# Convertir GeoDataFrame a GeoJSON
geojson = gdf.to_json()

In [ ]:
# Convertir CRS a WGS84
gdf = gdf.to_crs(epsg=4326)

In [ ]:
# Crear el mapa coroplético
fig = px.choropleth_mapbox(
    gdf,
    geojson=geojson,
    locations=gdf.index,
    color="tri1k_mean",
    hover_name="NOMGEO",
    mapbox_style="carto-positron",
    center={"lat": gdf.geometry.centroid.y.mean(), "lon": gdf.geometry.centroid.x.mean()},
    zoom=5,
    opacity=0.5
)

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
# Reproyectar a un CRS proyectado (por ejemplo, UTM Zone 14N, EPSG:32614)
gdf_projected = gdf.to_crs(epsg=32614)

# Calcular los centroides en el CRS proyectado
gdf_projected['centroid'] = gdf_projected.geometry.centroid

# Reproyectar de nuevo a WGS84
gdf = gdf_projected.to_crs(epsg=4326)
gdf['centroid'] = gdf['centroid'].to_crs(epsg=4326)

# Convertir GeoDataFrame a GeoJSON
geojson = gdf.to_json()

# Calcular el centro del mapa usando los centroides
center_lat = gdf['centroid'].y.mean()
center_lon = gdf['centroid'].x.mean()

In [ ]:
px.choropleth(gdf, color="tri1k_mean",geojson=geojson, locations=gdf.index, hover_name="NOMGEO", height=500)

In [ ]:
fig = px.choropleth_mapbox(
    data_frame = gdf.set_index("CVEGEO"), # Usar el ID como índice de los datos
    geojson = gdf.geometry,                 # La geometría
    locations = gdf.index,                  # El índice de los datos
    color = 'tri1k_mean',
    # Estética
    color_continuous_scale="Viridis",
    opacity=0.5,
    mapbox_style = 'open-street-map',
    center = {"lat": 23.4346697, "lon": -100.8707838},
    zoom = 3
)

In [ ]:
fig.show()